This tutorial notebook is intended to give you an overview of some mechanics of the game.

# Basics

With a `GameManager` object we can manipulate the game state as we desire.

In [ ]:
from core.game import GameManager

In [ ]:
?GameManager

In [ ]:
gm = GameManager()

We can build our own `board` and `state` or simply start with an existing one. 

In this case we use a scenario where we have 2 red tanks and 2 blue tanks.

In [ ]:
from core.scenarios import buildScenario

In [ ]:
?buildScenario

In [ ]:
board, state = buildScenario('Test2v2')

From the state, can we get our figures using the `getFigure()` method from a `GameState` object.

In [ ]:
from core.const import RED, BLUE

In [ ]:
?state.getFigure

In [ ]:
r1, r2 = state.getFigures(RED)
b1, b2 = state.getFigures(BLUE)

As an example, we want to attack `b1` with `r2` using `r2`'s cannon (`CA`).

To do so we build an `atk` of type `Attack` using the method `actionAttackFigure` from the `GameManger` object.

In [ ]:
?gm.actionAttackFigure

In [ ]:
try:
    atk = gm.actionAttackFigure(board, state, r2, b1, r1.weapons['CA'])
except ValueError as e:
    print(e)

...and we cannot do that since we don't have LOS on target. We need to first move `r2` to have a clean LOS on target.

This is the right time to start to _visualize_ the map
 we are playing on.

In [ ]:
from utils.images import drawState

In [ ]:
?drawState

In [ ]:
import utils.images
utils.images.SIZE = 18

We can globally change the size of the output images (I suggest to use multiples of 6 to avoid artifacts).

In [ ]:
drawState(board, state)

Much more clearer. Let's highlight the position of `r2` and `b1` on the map.

At the same time we draw the LOS from `r2` to `b1`.

In [ ]:
from utils.images import drawHexagon, drawLine

In [ ]:
?drawHexagon

In [ ]:
?drawLine

In [ ]:
img = drawState(board, state)
r2_pos = r2.position.tuple()
b1_pos = b1.position.tuple()
drawHexagon(img, r2_pos, '#ff0000', width=2)  # red hex
drawHexagon(img, b1_pos, '#0000ff', width=2)  # blue hex
drawLine(img, r2_pos, b1_pos, color='#00ff00', width=2) # green line
img

The LOS between `r2` and `b1` goes through a _dark grey_ hex. This is a blocker for the LOS.

We need to move `r2` in a position that can see `b1` to obtain a valid shoot. `r2` position is:

In [ ]:
r2.position

A triple means that it's current position is expressed in `cube-space`, we want coordinates in `grid-space` since it is much more natural for us. We just need to convert to `tuple`:

In [ ]:
r2.position.tuple()

The game works in cube-space so any position need to be expressed in `Cube`. Fortunately we can use the coordinates utility classes to to convert from `grid-space` (`Hex`) to `cube-space` (`Cube`).

In [ ]:
from core.utils.coordinates import Hex

In [ ]:
dst = Hex(5,8).cube()
m = gm.actionMove(board, state, r2, destination=dst)
m

Object `m` is our movement action from `(2,2)` (current position of `r1`) to `(4,4)`.

We can now generate a new state where the tank `r1` changed its position.

In [ ]:
new_state, outcome = gm.activate(board, state, m)

Note how `new_state` is not the same as previous `state`.

In [ ]:
new_state == state

We can check by looking at the images of the states:

In [ ]:
drawState(board, state)

In [ ]:
drawState(board, new_state)

Or we can directly update the original state.

In [ ]:
outcome = gm.step(board, state, m)

Now `new_state` and `state` are the same:

In [ ]:
new_state == state

In [ ]:
drawState(board, state)

The image also confirms the equality of the state.

We can now try to build the `Attack` action of before.

In [ ]:
atk = gm.actionAttackFigure(board, state, r2, b1, r1.weapons['CA'])

No error means success! We can execute the action.

In [ ]:
gm.step(board, state, atk)

It is also possible to draw actions on the map: when an action is executed and the state updated, the state keep this action in the `state.lastAction` field.

In [ ]:
drawState(board, state, True)

If we want to try a response, we first need to move `b1`:

In [ ]:
m = gm.actionMove(board, state, b2, destination=Hex(10,10).cube())
gm.step(board, state, m)

drawState(board, state, True)

In this situation, `r2` can respond to the last action of `b2`.

In [ ]:
res = gm.actionRespond(board, state, r2, b2, r1.weapons['CA'])
gm.step(board, state, res)

drawState(board, state, True)

# Agents

We create two agents: one for **red** and one for **blue**.

In [1]:
from agents import GreedyAgent


In [ ]:
red = GreedyAgent(RED, 42)
blue = GreedyAgent(BLUE, 24)

In order to play with the agents, we need to use the `MatchManager`object. We will recycle the `state` and `board` above.

In [ ]:
from agents import MatchManager

In [ ]:
board, state = buildScenario('Test2v2')

In [ ]:
mm = MatchManager('', red, blue, board, state, 2)

Since the `MatchManager` outputs a lot of information, it is useful to enable the logs.

In [ ]:
from utils.setup_logging import setup_logging
setup_logging()

With this object we can advance step by step in a match.

In [ ]:
mm.nextStep()
drawState(mm.board, mm.state, True)

In [ ]:
mm.nextStep()
drawState(mm.board, mm.state, True)

In [ ]:
mm.nextStep()
drawState(mm.board, mm.state)

In [ ]:
mm.nextStep()
drawState(mm.board, mm.state, True)

We can highlight a unit using drawing functions.

Or let the match play itself.

In [ ]:
mm.reset()
mm.play()